# Generate assesment

# Init

In [1]:
# Imports 
import os
from dotenv import load_dotenv, find_dotenv

# to disable SSL verification that causes problem from my laptop
from ssl_workaround import no_ssl_verification

# LangChain imports
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import SequentialChain
import langchain
langchain.debug = True
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
# formatters
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [2]:
# load secrets from env var
_ = load_dotenv(find_dotenv(),verbose=True) # read local .env file

# for Azure OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")
OPENAI_API_TYPE= os.getenv("OPENAI_API_TYPE")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
OPENAI_DEPLOYMENT_ID = os.getenv("OPENAI_DEPLOYMENT_ID")

# Chain 1

In [69]:
# Define response schema (not really needed, but for consistency)
step1_json = ResponseSchema(name="step-1",
                             description="step 1 reasoning")
step2_json = ResponseSchema(name="step-2",
                             description="step 2 reasoning")
step3_json = ResponseSchema(name="step-3",
                             description="step 3 reasoning")
context_json = ResponseSchema(name="context",
                             description="generated context paragraph")
frq_json = ResponseSchema(name="frq",
                             description="generated free respoce question")


response_schemas = [step1_json,step2_json,step3_json,context_json,frq_json]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
fmt_instr_frq = output_parser.get_format_instructions()
print(fmt_instr_frq)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"step-1": string  // step 1 reasoning
	"step-2": string  // step 2 reasoning
	"step-3": string  // step 3 reasoning
	"context": string  // generated context paragraph
	"frq": string  // generated free respoce question
}
```


In [70]:
system_message = """
You are an helpfull assistant. User will provide you Common Core State Standard and a topic. \
You will output an assesment task for a student to test student knowledge consisting of:
 1. COTEXT: context paragraph that will be used by the student to answer the below free responce question.\
 The COTEXT is a paragraph and must be related to topic provided by user. \
 The COTEXT must be sufficient to answer the free responce question and meet complexity and length as expected by the Common Core State Standard definition provided by user.
 2. FREE RESPONCE QUESTION: The question must be related to the context article and test students ability as per the standard definition above.\

Follow these steps to create the task:

Step 1: Provide definition of Common Core State Standard provided by user

Step 3: Define what needs to be tested as part of assement defined in steps 1 and make sure to include it in the CONTEXT and FREE RESPONSE QUESTION.

Step 2: Define what is the expected lenght and complexity of the CONTEXT paragraph that is provided to student

Step 4: Generate CONTEXT paragraph. It must meet the conditions defined in second and third step.

Step 5: Generate FREE RESPONSE QUESTION related to CONEXT and meeting definition in step one.

{fmt_instr_frq}
"""

In [71]:
# Make SystemMessagePromptTemplate
prompt=PromptTemplate(
    template=system_message,
    input_variables=["fmt_instr_frq"]
)

system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

In [72]:
CCSS_TOPIC = """Common Core State Standard is {standard} and topic is {topic}."""

In [73]:
# Make HumanMessagePromptTemplate
human_message_prompt = HumanMessagePromptTemplate.from_template(CCSS_TOPIC)

In [74]:
# Create ChatPromptTemplate: Combine System + Human
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt

ChatPromptTemplate(input_variables=['fmt_instr_frq', 'standard', 'topic'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fmt_instr_frq'], output_parser=None, partial_variables={}, template='\nYou are an helpfull assistant. User will provide you Common Core State Standard and a topic. You will output an assesment task for a student to test student knowledge consisting of:\n 1. COTEXT: context paragraph that will be used by the student to answer the below free responce question. The COTEXT is a paragraph and must be related to topic provided by user.  The COTEXT must be sufficient to answer the free responce question and meet complexity and length as expected by the Common Core State Standard definition provided by user.\n 2. FREE RESPONCE QUESTION: The question must be related to the context article and test students ability as per the standard definition above.\nFollow these steps to create the task:\n\nStep 1: P

In [75]:
chain1 = LLMChain(
    llm=ChatOpenAI(
        temperature=0.7, 
        openai_api_base=OPENAI_API_BASE,
        openai_api_key=OPENAI_API_KEY,
        deployment_id = OPENAI_DEPLOYMENT_ID),
    prompt = chat_prompt,
    output_key="reasoning_context_frq"
)

WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.


In [76]:
chain1
with no_ssl_verification():
    response = chain1.predict(standard = "CCSS.ELA-LITERACY.W.4.9", topic='ocean', fmt_instr_frq = fmt_instr_frq)

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "standard": "CCSS.ELA-LITERACY.W.4.9",
  "topic": "ocean",
  "fmt_instr_frq": "The output should be a markdown code snippet formatted in the following schema, including the leading and trailing \"```json\" and \"```\":\n\n```json\n{\n\t\"step-1\": string  // step 1 reasoning\n\t\"step-2\": string  // step 2 reasoning\n\t\"step-3\": string  // step 3 reasoning\n\t\"context\": string  // generated context paragraph\n\t\"frq\": string  // generated free respoce question\n}\n```"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: \nYou are an helpfull assistant. User will provide you Common Core State Standard and a topic. You will output an assesment task for a student to test student knowledge consisting of:\n 1. COTEXT: context paragraph that will be used by the student to answer the below free responce question. The COTEXT is a paragraph and must be related to

[llm/end] [1:chain:LLMChain > 2:llm:ChatOpenAI] [6.34s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{\n\t\"step-1\": \"The Common Core State Standard CCSS.ELA-LITERACY.W.4.9 focuses on the ability of a student to draw evidence from literary or informational texts to support analysis, reflection, and research. This standard emphasizes the importance of using texts to support arguments, interpretations, and explanations.\",\n\t\"step-2\": \"The context paragraph must be at least 4-5 sentences long and should be written at a 4th-grade reading level. It should provide enough information about the ocean to allow the student to answer the free response question.\",\n\t\"step-3\": \"The assessment should test the student's ability to draw evidence from the context paragraph to support their response to the free response question.\",\n\t\"context\": \"The ocean covers more than 70% of the Earth's surface and is home to a diverse range of plant and 

In [77]:
print(response)

```json
{
	"step-1": "The Common Core State Standard CCSS.ELA-LITERACY.W.4.9 focuses on the ability of a student to draw evidence from literary or informational texts to support analysis, reflection, and research. This standard emphasizes the importance of using texts to support arguments, interpretations, and explanations.",
	"step-2": "The context paragraph must be at least 4-5 sentences long and should be written at a 4th-grade reading level. It should provide enough information about the ocean to allow the student to answer the free response question.",
	"step-3": "The assessment should test the student's ability to draw evidence from the context paragraph to support their response to the free response question.",
	"context": "The ocean covers more than 70% of the Earth's surface and is home to a diverse range of plant and animal life. It is divided into five main basins: the Atlantic, Pacific, Indian, Southern, and Arctic. The ocean plays a crucial role in regulating the Earth's c

# Chain 2

In [78]:
# Define response schema (not really needed, but for consistency)
rubric_json = ResponseSchema(name="rubric",
                             description="Rubric for this assesment task as expected by the Common Core State Standard definition provided by user")

response_schemas = [rubric_json]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
fmt_instr_rubric = output_parser.get_format_instructions()
print(fmt_instr_rubric)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"rubric": string  // Rubric for this assesment task as expected by the Common Core State Standard definition provided by user
}
```


In [79]:
system_message = """
You are an helpfull assistant. User will provide you Common Core State Standard. \
You will also be provided with CONTEXT and FREE RESPONSE QUESTION.
You will output a rubric for this assesment task as expected by the Common Core State Standard definition provided by user.
{fmt_instr_rubric}
"""
# Follow these steps to create the task:

# Step 1:{delimiter} Provide definition of Common Core State Standard provided by user

# Step 3:{delimiter} Define what needs to be tested as part of assement defined in steps 1 and make sure to include it in the CONTEXT and FREE RESPONSE QUESTION.

# Step 2:{delimiter} Define what is the expected lenght and complexity of the CONTEXT paragraph that is provided to student

# Step 4:{delimiter} Generate CONTEXT paragraph. It must meet the conditions defined in second and third step.

# Step 5:{delimiter} Generate FREE RESPONSE QUESTION related to CONEXT and meeting definition in step one.

# Use the following format:
# Step 1:{delimiter} <step 1 reasoning>
# Step 2:{delimiter} <step 2 reasoning>
# Step 3:{delimiter} <step 3 reasoning>
# Step 4 - CONTEXT:{delimiter} <generated contex paragraph>
# Step 5 - FREE RESPONCE QUESTION:{delimiter} <generated free respoce question>

# Make sure to include four hashtags to delimit each step,\
# i.e. {delimiter}. 
# """

In [80]:
# Make SystemMessagePromptTemplate
prompt=PromptTemplate(
    template=system_message,
    input_variables=["fmt_instr_rubric"]
)

system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)


In [81]:
CCSS_RUBRIC = """Common Core State Standard is {standard}. 
CONTEXT and FREE RESPONSE QUESTION are included in step 4 and 5 {reasoning_context_frq}."""

In [82]:
# Make HumanMessagePromptTemplate
human_message_prompt = HumanMessagePromptTemplate.from_template(CCSS_RUBRIC)

In [83]:
# Create ChatPromptTemplate: Combine System + Human
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt

ChatPromptTemplate(input_variables=['fmt_instr_rubric', 'reasoning_context_frq', 'standard'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fmt_instr_rubric'], output_parser=None, partial_variables={}, template='\nYou are an helpfull assistant. User will provide you Common Core State Standard. You will also be provided with CONTEXT and FREE RESPONSE QUESTION.\nYou will output a rubric for this assesment task as expected by the Common Core State Standard definition provided by user.\n{fmt_instr_rubric}\n', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['reasoning_context_frq', 'standard'], output_parser=None, partial_variables={}, template='Common Core State Standard is {standard}. \nCONTEXT and FREE RESPONSE QUESTION are included in step 4 and 5 {reasoning_context_frq}.', template_format='f-string', validate_template=Tr

In [84]:
chain2 = LLMChain(
    llm=ChatOpenAI(
        temperature=0.0, 
        openai_api_base=OPENAI_API_BASE,
        openai_api_key=OPENAI_API_KEY,
        deployment_id = OPENAI_DEPLOYMENT_ID),
    prompt = chat_prompt,
    output_key="rubric"
)

WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.


In [85]:
overall_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=["standard","topic","fmt_instr_rubric","fmt_instr_frq"],
    output_variables=["rubric","reasoning_context_frq"],
    verbose=True
)

with no_ssl_verification():
    solution = overall_chain({"standard":"CCSS.ELA-LITERACY.W.4.9", "topic":"desert", "fmt_instr_rubric":fmt_instr_rubric, "fmt_instr_frq":fmt_instr_frq})

[chain/start] [1:chain:SequentialChain] Entering Chain run with input:
{
  "standard": "CCSS.ELA-LITERACY.W.4.9",
  "topic": "desert",
  "fmt_instr_rubric": "The output should be a markdown code snippet formatted in the following schema, including the leading and trailing \"```json\" and \"```\":\n\n```json\n{\n\t\"rubric\": string  // Rubric for this assesment task as expected by the Common Core State Standard definition provided by user\n}\n```",
  "fmt_instr_frq": "The output should be a markdown code snippet formatted in the following schema, including the leading and trailing \"```json\" and \"```\":\n\n```json\n{\n\t\"step-1\": string  // step 1 reasoning\n\t\"step-2\": string  // step 2 reasoning\n\t\"step-3\": string  // step 3 reasoning\n\t\"context\": string  // generated context paragraph\n\t\"frq\": string  // generated free respoce question\n}\n```"
}
[chain/start] [1:chain:SequentialChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "standard": "CCSS.ELA-LITERAC

In [86]:
print(solution)

{'standard': 'CCSS.ELA-LITERACY.W.4.9', 'topic': 'desert', 'fmt_instr_rubric': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"rubric": string  // Rubric for this assesment task as expected by the Common Core State Standard definition provided by user\n}\n```', 'fmt_instr_frq': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"step-1": string  // step 1 reasoning\n\t"step-2": string  // step 2 reasoning\n\t"step-3": string  // step 3 reasoning\n\t"context": string  // generated context paragraph\n\t"frq": string  // generated free respoce question\n}\n```', 'rubric': '```json\n{\n\t"rubric": "The student\'s response will be assessed based on their ability to:\\n- Introduce the topic clearly\\n- Group related information into paragraphs\\n- Use linking words and phrases to conne

FIX this above !!!!!!!!!!!!!!!!!!!

In [87]:
def get_output(answer,item):
    return output_parser.parse(answer[item])

In [88]:
get_output(solution,"rubric")

{'rubric': "The student's response will be assessed based on their ability to:\n- Introduce the topic clearly\n- Group related information into paragraphs\n- Use linking words and phrases to connect ideas within categories of information\n- Provide a concluding statement or section related to the information or explanation presented\n- Include accurate and relevant information about how plants and animals have adapted to life in the desert\n- Use linking words and phrases to connect ideas and group related information into paragraphs\n- Use appropriate vocabulary and descriptive language to convey ideas clearly\n- Demonstrate a clear and concise writing style\n- Follow conventions of standard English grammar, usage, capitalization, punctuation, and spelling"}

In [89]:
get_output(solution,"frq")

KeyError: 'frq'